In [ ]:
from collections import deque # Used to implement queues.
import random # Random choice, etc.
import string
import heapq # Used in discrete event simulator
import numpy as np # Used for gamma probability distribution, and percentiles.

def fmt(x):
    """Formats a number x which can be None, for convenience."""
    return None if x is None else "{:.2f}".format(x)

class Event(object):

    def __init__(self, method, delay=0, args=None, kwargs=None):
        """An event consists in calling a specified method after a delay,
        with a given list of args and kwargs."""
        self.method = method
        self.delay = delay
        self.args = args or []
        self.kwargs = kwargs or {}
        self.time = None # Not known until this is added to the queue.

    def __call__(self, time):
        """This causes the event to happen, returning a list of new events
        as a result. Informs the object of the time of occurrence."""
        return self.method(*self.args, time=time, **self.kwargs)

    def __lt__(self, other):
        return self.time < other.time

    def __repr__(self):
        return "@{}: {} {} {} {} dt={:.2f}".format(
            fmt(self.time),
            self.method.__self__.__class__.__name__,
            self.method.__name__,
            self.args, self.kwargs, self.delay
        )

class EventSimulator(object):

    def __init__(self, trace=False):
        self.events = []
        self.time = 0 # This is the global time.
        self.trace = trace

    def add_event(self, event):
        """Adds an event to the queue."""
        event.time = self.time + event.delay
        heapq.heappush(self.events, event)

    def step(self):
        """Performs a step of the simulation."""
        if len(self.events) > 0:
            event = heapq.heappop(self.events)
            self.time = event.time
            new_events = event(self.time) or []
            for e in new_events:
                self.add_event(e)
            if self.trace:
                print("Processing:", event)
                print("New events:", new_events)
                print("Future events:", self.events)

    def steps(self, number=None):
        """Performs at most number steps (or infinity, if number is None)
        in the simulation."""
        num_done = 0
        while len(self.events) > 0:
            self.step()
            num_done += 1
            if num_done == number:
                break

class Stop(object):

    def __init__(self):
        self.num_people = 0

    # Event method
    def person_arrives(self, time=None):
        # We increment the number of people.
        self.num_people += 1
        # and we schedule the next arrival.
        return [Event(self.person_arrives, delay=np.random.exponential())]

    def get_everybody(self):
        """Returns the number of people who climbs on the bus, and sets
        the people at the station back to 0."""
        n = self.num_people
        self.num_people = 0
        return n

class Bus(object):

    def __init__(self, stop):
        self.stop = stop
        self.passengers = []

    # Event method
    def arrives(self, time=None):
        self.passengers.append(self.stop.get_everybody())
        return [Event(self.arrives, delay=np.random.gamma(2, 10))]

stop = Stop()
bus = Bus(stop)

es = EventSimulator()

# We seed the initial events.
es.add_event(Event(bus.arrives, delay=np.random.gamma(2, 10)))
es.add_event(Event(stop.person_arrives, delay=np.random.exponential()))

# We run 100 steps.
es.steps(number=100)

# And we list how many passengers climbed on the bus each time it stopped.
bus.passengers


class Person(object):

    def __init__(self, start_time=None, num_things=1):
        """Creates a person.
        @param time: time at which person enters the system.
        @param delay_scale: average delay - this is the scale of the gamma
            distribution."""
        self.start_time = start_time # Time of entering the system
        self.end_time = None
        self.num_things = num_things

    # Event method
    def finish(self, time=None):
        """The person has finished the iter through the network."""
        self.end_time = time
        return [] # No events generated as a consequence.

    @property
    def elapsed_time(self):
        return None if self.end_time is None else self.end_time - self.start_time

    def __repr__(self):
        return "Person: D{:.2f} {}-{} elapsed={}".format(
            self.delay_scale,
            fmt(self.start_time), fmt(self.end_time), fmt(self.elapsed_time))


class Source(object):
    """Source of people, to put into a plaza."""

    def __init__(self, rate=1, slow_fraction=5, num_things=1, plaza=None, number=None):
        """Creates people at a certain rate.  Puts them into the plaza.
        @param slow_fraction: fraction of slow people.
        @param rate: rate at which people are entering.
        @param plaza: plaza where people enter.
        @param number: how many people to generate; None = infinity.
        """
        self.plaza = plaza
        self.slow_fraction = slow_fraction
        self.rate = rate
        self.number = number
        self.num_things = num_things

    def start(self, time=None):
        """Starts generating people."""
        if self.number == 0:
            return [] # Nothing more to be done.
        num_things = self.num_things
        if np.random.random() < 1 / self.slow_fraction:
            num_things *= self.slow_fraction
        # Creates the person
        person = Person(num_things=num_things, start_time=time)
        enter_event = Event(self.plaza.enter, args=[person])
        # Schedules the next person creation.
        self.number = None if self.number is None else self.number - 1
        dt = np.random.gamma(1, 1/self.rate)
        start_event = Event(self.start, delay=dt)
        return [enter_event, start_event]

class Plaza(object):
    """A plaza is a place where people can decide which queue to join next.
    In a random plaza, people simply choose a random queue."""

    def __init__(self, queues=None):
        """Creates a plaza, with an optional attached set of queues.
        The queues can be left to empty, and registered later."""
        self.queues = queues or [] # Queues attached to the plaza.
        # We keep track of who went through it.
        self.visitors = []

    def register_queue(self, queue):
        if queue not in self.queues:
            self.queues.append(queue)

    # Event method
    def enter(self, person, time=None):
        """When a person enters the plaza, a new event is generated,
        of the person entering the shortest queue.
        We could have also have a "dark plaza" subclass in which the
        person enters a randomly selected queue instead."""
        self.visitors.append(person)
        if len(self.queues) == 0:
            # If there are no queues connected to the plaza,
            # the plaza is an exit.
            person.finish(time=time)
            return [] # No events generated.
        len_queues = [len(q) for q in self.queues]
        shortest = min(len_queues)
        q = random.choice([q for q in self.queues if len(q) == shortest])
        return [Event(q.enter, args=[person])]


class Queue(object):

    def __init__(self):
        """We create a queue.  The workers will be filled in later, by calls
        to register_worker from the workers."""
        self.people = deque()
        self.workers = []

    def register_worker(self, worker):
        """This is called by a worker to connect with the queue."""
        if worker not in self.workers:
            self.workers.append(worker)

    # Event method
    def enter(self, person, time=None):
        """Appends the person to the queue, and alerts all the workers,
        by generating `notify` events for each of them."""
        self.people.append(person)
        # Notifies the workers that now there is someone in the queue.
        new_events = []
        for worker in self.workers:
            new_events.append(Event(worker.notify))
        # These are the notification events.
        return new_events

    def exit(self):
        """Picks a person out of the queue, or None if the
        queue is empty."""
        return self.people.popleft() if len(self.people) > 0 else None

    def __len__(self):
        return len(self.people)

    @property
    def empty(self):
        return len(self.people) == 0


class Worker(object):

    def __init__(self, plaza=None, delay_factor=None, queue=None):
        """Creates a worker, optionally passing to it the queues.
        @param plaza: plaza where one goes after being done with the worker.
        @param delay_shape: shape factor for delay distribution. The scale
            comes from the person.
        @param queue: connected queue.
        """
        # Connects the queue and the worker, ...
        self.queue = queue
        queue.register_worker(self)
        # ... and the worker with the plaza.
        self.plaza = plaza
        self.busy = False # Initially, not serving anyone.
        self.delay_factor = delay_factor

    def _get_person_from_queue(self):
        """Gets a person from the queue, and returns the list of
        events to be scheduled as a consequence (None, if no person
        was in fact present, or the finish event for this worker)."""
        person = self.queue.exit()
        if person is None:
            return []
        self.busy = True
        dt = np.random.gamma(self.delay_factor, scale=person.num_things)
        return [Event(self.finish, delay=dt, args=[person])]

    # Event method
    def notify(self, time=None):
        """The queue is telling the worker that there's some job in it."""
        return [] if self.busy else self._get_person_from_queue()

    # Event method
    def finish(self, person, time=None):
        """A person is done with the worker. The person enters the plaza,
        and a new person, if any is available, is taken from one of the queues."""
        # The person will enter the plaza.
        new_events = [Event(self.plaza.enter, args=[person])]
        # We check if there is someone in line.
        self.busy = False # We change to True if needed.
        if len(self.queue) > 0:
            new_events.extend(self._get_person_from_queue())
        return new_events


import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

def histogram_time(plaza):
    """Given a plaza, plots the histogram of time it took people
    in that plaza to go through the network."""
    points = [p.elapsed_time for p in plaza.visitors]
    plt.hist(points)
    plt.show()

def time90(plaza, q=90):
    """Returns the 90% percentile of the time, so that 90% of people
    complete within that time.  This for any 90% q."""
    return np.percentile([p.elapsed_time for p in plaza.visitors], q)


import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

def histogram_time(plaza):
    """Given a plaza, plots the histogram of time it took people
    in that plaza to go through the network."""
    points = [p.elapsed_time for p in plaza.visitors]
    plt.hist(points)
    plt.show()

def time90(plaza, q=90):
    """Returns the 90% percentile of the time, so that 90% of people
    complete within that time.  This for any 90% q."""
    return np.percentile([p.elapsed_time for p in plaza.visitors], q)

source_rate = 0.1
num_things = 5
source_number = 10000
num_steps = 200000
source_slow_fraction = 5


entrance = Plaza()
wayout = Plaza()
s = Source(rate=source_rate, slow_fraction=source_slow_fraction,
           number=source_number, plaza=entrance)

q1 = Queue()
q2 = Queue()
q3 = Queue()
w1 = Worker(delay_factor=2, queue=q1, plaza=wayout)
w2 = Worker(delay_factor=3, queue=q2, plaza=wayout)
w3 = Worker(delay_factor=4, queue=q3, plaza=wayout)

entrance.register_queue(q1)
entrance.register_queue(q2)
entrance.register_queue(q3)

e = Event(s.start)

sim = EventSimulator(trace=False)
sim.add_event(e)
sim.steps(num_steps)

print("Still in:", source_number - len(wayout.visitors))
print("95% time:", time90(wayout, q=95))
histogram_time(wayout)

entrance = Plaza()
wayout = Plaza()
q = Queue()
w1 = Worker(delay_factor=2, queue=q, plaza=wayout)
w2 = Worker(delay_factor=3, queue=q, plaza=wayout)
w3 = Worker(delay_factor=4, queue=q, plaza=wayout)
s = Source(rate=source_rate, slow_fraction=source_slow_fraction,
           number=source_number, plaza=entrance)
entrance.register_queue(q)

e = Event(s.start)

sim = EventSimulator(trace=False)
sim.add_event(e)
sim.steps(num_steps)

print("Still in:", source_number - len(wayout.visitors))
print("95% time:", time90(wayout, q=95))
histogram_time(wayout)


source_rate = 0.1
source_slow_fraction = 10
source_number = 10000
num_steps = 200000


#####
entrance = Plaza()
wayout = Plaza()

# Three intermediate plazas.
pl1 = Plaza()
pl2 = Plaza()
pl3 = Plaza()

# These are queues.
qa1 = Queue()
qa2 = Queue()
qa3 = Queue()
qb1 = Queue()
qb2 = Queue()
qb3 = Queue()

entrance.register_queue(qa1)
entrance.register_queue(qa2)
entrance.register_queue(qa3)
pl1.register_queue(qb1)
pl2.register_queue(qb2)
pl3.register_queue(qb3)

# Workers for the first stage.
w1 = Worker(delay_factor=2, queue=qa1, plaza=pl1)
w2 = Worker(delay_factor=3, queue=qa2, plaza=pl2)
w3 = Worker(delay_factor=4, queue=qa3, plaza=pl3)

# Workers for the second stage.
v1 = Worker(delay_factor=3, queue=qb1, plaza=wayout)
v2 = Worker(delay_factor=4, queue=qb2, plaza=wayout)
v3 = Worker(delay_factor=5, queue=qb3, plaza=wayout)

# Source
s = Source(rate=source_rate, slow_fraction=source_slow_fraction,
           number=source_number, plaza=entrance)

e = Event(s.start)

sim = EventSimulator(trace=False)
sim.add_event(e)
sim.steps(num_steps)

print("Still in:", source_number - len(wayout.visitors))
print("95% time:", time90(wayout, q=95))
histogram_time(wayout)


entrance = Plaza()
wayout = Plaza()

# These are the joint queues.
qa = Queue()
qb = Queue()

# One intermediate plaza.
pl = Plaza()

entrance.register_queue(qa)
pl.register_queue(qb)

# Workers for the first stage.
w1 = Worker(delay_factor=2, queue=qa, plaza=pl)
w2 = Worker(delay_factor=3, queue=qa, plaza=pl)
w3 = Worker(delay_factor=4, queue=qa, plaza=pl)

# Workers for the second stage.
v1 = Worker(delay_factor=3, queue=qb, plaza=wayout)
v2 = Worker(delay_factor=4, queue=qb, plaza=wayout)
v3 = Worker(delay_factor=5, queue=qb, plaza=wayout)

# Source
s = Source(rate=source_rate, slow_fraction=source_slow_fraction,
           number=source_number, plaza=entrance)

e = Event(s.start)

sim = EventSimulator(trace=False)
sim.add_event(e)
sim.steps(num_steps)

print("Still in:", source_number - len(wayout.visitors))
print("95% time:", time90(wayout, q=95))
histogram_time(wayout)
